## Analysis of snRNA-seq data from young healthy mice

### Libraries, figure parameters and custom functions

In [ ]:
import scanpy as sc
import numpy as np
import pandas as pd
import anndata as ad
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib import colors
from matplotlib import rcParams
from gprofiler import gprofiler
import os
from scipy import stats
import custom_functions as cf

Set parameters for plots in scanpy

In [ ]:
sc.settings.set_figure_params(dpi=80)

sc.set_figure_params(scanpy=True, dpi=80, dpi_save=250,
                     frameon=True, vector_friendly=True,
                     color_map="YlGnBu", format='pdf', transparent=False,
                     ipython_format='png2x')

### Data

Load the expression matrix

In [ ]:
adata = ad.read("/home/maria/data/polyploid_hepatocytes/matrices/snRNAseq_mouse_hepatocytes_young_may20_raw_with_metadata.h5ad")

Manually adding the raw counts of some sparsely expressed marker genes

In [ ]:
adata.obs["Krt14"] = adata[:,"ENSMUSG00000045545"].X
adata.obs["Krt19"] = adata[:,"ENSMUSG00000020911"].X
adata.obs["Krt7"] = adata[:,"ENSMUSG00000023039"].X
adata.obs["Krt9"] = adata[:,"ENSMUSG00000051617"].X
adata.obs["Prom1"] = adata[:,"ENSMUSG00000029086"].X
adata.obs["Dlk1"] = adata[:,"ENSMUSG00000040856"].X
adata.obs["Erbb2"] = adata[:,"ENSMUSG00000062312"].X
adata.obs["Mki67"] = adata[:,"ENSMUSG00000031004"].X
adata.obs["Oct4"] = adata[:,"ENSMUSG00000024406"].X
adata.obs["Cd34"] = adata[:,"ENSMUSG00000016494"].X
adata.obs["Tpm2"] = adata[:,"ENSMUSG00000028464"].X
adata.obs["Ace2"] = adata[:,"ENSMUSG00000015405"].X

In [ ]:
# split ERCC counts into a different object
ERCC = []
for i in adata.var_names:
    if i[0:3] != "ENS" and i[0:2] != "__":
        ERCC.append(i)
        
adataERCC = adata[:,ERCC].copy()

In [ ]:
# split endogenous transcript counts into a different object
transc = []
for i in adata.var_names:
    if i[0:3] == "ENS":
        transc.append(i)

adataT = adata[:,transc].copy()

Removing additional output from htseq-count

In [ ]:
# remove the additional output from htseq-count from the matrix
amb = []
for i in adata.var_names:
    if i[0:2] != "__":
        amb.append(i)
        
adata = adata[:,amb].copy()

In [ ]:
adata.obs['n_counts_raw'] = adata.X.sum(axis=1) # endogeneous transcripts + ERCCs
adata.obs["n_counts_transcripts"] = np.sum(adataT.X, axis=1) # endogenous transcripts only
adata.obs["n_counts_ERCC"] = np.sum(adataERCC.X, axis=1) # ERCCs only
adata.obs["percentERCC"] = pd.to_numeric(adata.obs["percentERCC"]) # make percent ERCC column numeric

Plotting transcript read counts, intronic read counts, exonic read counts, and ribosobal RNA read counts

In [ ]:
df = pd.DataFrame()
df["transcripts"] = adata.obs["n_counts_transcripts"]
df["introns"] = adata.obs["intron_counts"]
df["exons"] = adata.obs["exon_counts"]
df["rrna"] = adata.obs["rrna"]

In [ ]:
t = np.mean(df["transcripts"])
i = np.mean(df["introns"])
e = np.mean(df["exons"])
r = np.mean(df["rrna"])

In [ ]:
x = np.arange(4)
variable = [t, i , e, r]

fig, ax = plt.subplots()
plt.bar(x, variable)
plt.xticks(x, ('transcripts', 'introns', 'exons', 'rRNA'))
ax.set_xticklabels(ax.get_xticklabels(), rotation=45, horizontalalignment='right')
plt.show()

In [ ]:
np.mean(df["introns"])/np.mean(df["transcripts"])

In [ ]:
np.mean(df["exons"])/np.mean(df["transcripts"])

In [ ]:
np.mean(df["rrna"])/np.mean(df["transcripts"])

### Cutoff visualization

In [ ]:
# visualize cutoffs for filtering cells based on their percentage of ERCC reads
p = plt.hist(adata.obs["percentERCC"], bins=100, color='c', edgecolor='k', alpha=0.65)
plt.axvline(0.9, color='k', linestyle='dashed', linewidth=1)
plt.axvline(0.05, color='k', linestyle='dashed', linewidth=1)

In [ ]:
# remove cells that have no genes expressed in order to better visualize the cutoffs
sc.pp.filter_cells(adata, min_genes=1)
sc.pp.filter_cells(adata, min_counts=1)

In [ ]:
# visualize lower cutoff for filtering cells based on the number of genes
p = plt.hist(adata.obs["n_genes"], bins=100, color='c', edgecolor='k', alpha=0.65)
plt.axvline(1000, color='k', linestyle='dashed', linewidth=1)

In [ ]:
adata

### Filtering

Keeping nuclei with 5-90% ERCC reads and a minimum of 1000 genes expressed  
Keeping genes that are present in at least 25 cells with at least 250 overall reads across cells

In [ ]:
adata_new = adata[adata.obs["percentERCC"] <= 0.9,:].copy()
adata_new = adata_new[adata_new.obs["percentERCC"] > 0.05,:].copy()

sc.pp.filter_cells(adata_new, min_genes=1000) 
sc.pp.filter_genes(adata_new, min_cells=25) # roughly 1% of the population
sc.pp.filter_genes(adata_new, min_counts=250)

In [ ]:
adata_new

In [ ]:
## further filtering: 
adata2 = adata_new.copy()

# remove cells with 2x2n and - ploidy (doublets and empty wells from FACS failure)
adata2 = adata2[adata2.obs['Ploidy'] != '2-2n', :]
adata2 = adata2[adata2.obs['Ploidy'] != '-', :]

# remove cells with more than 7000 genes covered
adata2 = adata2[adata2.obs['n_genes'] < 7000, :]

# remove cells with less than 10,000 transcript counts
adata2 = adata2[adata2.obs['n_counts_transcripts'] > 10000, :]
# remove cells with more than 300,000 transcript counts
adata2 = adata2[adata2.obs['n_counts_transcripts'] < 300000, :]

In [ ]:
adata2

In [ ]:
p = plt.hist(adata2.obs["n_counts_transcripts"], bins=100, color='c', edgecolor='k', alpha=0.65)

In [ ]:
sc.pl.violin(adata2, ['n_genes', 'n_counts_raw', 'percentERCC', "n_counts_ERCC", "n_counts_transcripts"], jitter=0.4, multi_panel=True)

In [ ]:
sc.pl.violin(adata2, "n_genes", groupby="Ploidy")

In [ ]:
np.median(adata2[adata2.obs["Ploidy"]=="4n"].obs["n_genes"])/np.median(adata2[adata2.obs["Ploidy"]=="2n"].obs["n_genes"])

In [ ]:
# number of genes per cell for comarison to other methods
df = pd.DataFrame()
df["n_genes"] = adata2.obs["n_genes"]
df["dataset"] = "snRNAseq2"
df.to_csv("/home/maria/data/polyploid_hepatocytes/comparing_ds/our_data_filtered_20200513.csv")

### ERCC size factor calculation

In [ ]:
ERCC = []
for i in adata2.var_names:
    if i[0:3] != "ENS" and i[0:2] != "__":
        ERCC.append(i)

In [ ]:
# separate the matrix into the 2 dilutions and calculate the ERCC size factor separately 
adataHD = adata2[adata2.obs["ERCC.dilution"] == "1 in 300.000",:].copy()
adataLD = adata2[adata2.obs["ERCC.dilution"] == "1 in 100.000",:].copy()
adataHD.obs["ERCC_size_factor"] = np.sum(adataHD[:,ERCC].X, axis=1)/np.mean(np.sum(adataHD[:,ERCC].X, axis=1))
adataLD.obs["ERCC_size_factor"] = np.sum(adataLD[:,ERCC].X, axis=1)/np.mean(np.sum(adataLD[:,ERCC].X, axis=1))

In [ ]:
adata2 = ad.AnnData.concatenate(adataLD, adataHD)
new_names = adataLD.obs_names.append(adataHD.obs_names)
adata2.obs_names= new_names

### Finally remove ERCC reads from the matrix

In [ ]:
red_trans = []
for i in adata2.var_names:
    if i[0:3] == "ENS":
        red_trans.append(i) # endogeneous transcripts
        
adata2 = adata2[:,red_trans].copy()        

Store raw counts as a layer

In [ ]:
adata2.layers["counts"] = adata2.X.copy()

### Gene length and size factor normalization

In [ ]:
# divide each column by transcript length (per 1kb)
for j in range(0, len(adata2.var)):
    adata2.X[:,j] = adata2.X[:,j]/(adata2.var["length"][j]/1000) 

adata2.obs['n_counts_TPM'] = adata2.X.sum(axis=1)

# sum the gene length-corrected counts per cell and divide by 10,000 times the ERCC size factor
adata2.obs['RPK_factor'] = adata2.obs['n_counts_TPM']/(adata2.obs['ERCC_size_factor']*10000)

# divide each row by the size factor
for i in range(0,len(adata2)):
     adata2.X[i,:] = adata2.X[i,:]/adata2.obs["RPK_factor"][i]
adata2.obs['n_counts_TPM_norm'] = adata2.X.sum(axis=1)

In [ ]:
# plotting the normalized counts per cell
p = plt.hist(adata2.obs["n_counts_TPM_norm"], bins=100, color='c', edgecolor='k', alpha=0.65)

Store normalized counts as layer

In [ ]:
adata2.layers["norm_counts"] = adata2.X.copy()

In [ ]:
sc.pl.violin(adata2, ['n_genes', 'n_counts_TPM'], jitter=.4, multi_panel=True)

Additional filtering: removing cells with high gene length-corrected counts

In [ ]:
adata2 = adata2[adata2.obs["n_counts_TPM"] <= 50000,:].copy()

In [ ]:
# remove the technical replicates
adata2 = adata2[adata2.obs["exp."] != "SNI-234(R2)",:].copy()
adata2 = adata2[adata2.obs["exp."] != "SNI-235(R2)",:].copy()

In [ ]:
# mean and median transcript count after filtering and normalization
adata2.obs["mean_transcript_count_TPM"] = np.mean(adata2.X, axis=1)
adata2.var["mean_exp_TPM"] = np.mean(adata2.X, axis=0)
adata2.var["median_exp_TPM"] = np.median(adata2.X, axis=0)

In [ ]:
adata2

Logarithmize

In [ ]:
sc.pp.log1p(adata2)

Store logarithmized data as adata.raw and as additional layer

In [ ]:
adata2.raw = adata2
adata2.layers["raw"] = adata2.X

Store binary expression as additional layer

In [ ]:
adata3 = adata2.copy()
adata3.X[adata3.X != 0] = 1
adata2.layers["raw_bin"] = adata3.X

### Correcting batch effect using combat

In [ ]:
sc.pp.combat(adata2, key="exp.")

### Visualisation

In [ ]:
sc.pp.pca(adata2)
sc.pp.neighbors(adata2)
sc.tl.umap(adata2)

In [ ]:
# PCA
sc.pl.pca_scatter(adata2, color=["Ploidy","n_genes", "ERCC.dilution", "exp."], wspace=0.4)

In [ ]:
# t-SNE
sc.tl.tsne(adata2, perplexity=30, n_pcs=15)
sc.pl.tsne(adata2, color=[ "Ploidy", "n_genes", "ERCC.dilution", "exp."], wspace=0.4)

In [ ]:
adata2

### Save adata for downstream analysis

In [ ]:
adata2.write('/home/maria/data/polyploid_hepatocytes/matrices/snRNAseq_mouse_hepatocytes_young_cells_filtered_TPM_may20.h5ad')

In [ ]:
adata = ad.read('/home/maria/data/polyploid_hepatocytes/matrices/snRNAseq_mouse_hepatocytes_young_cells_filtered_TPM_may20.h5ad')

### Cell type identification

In [ ]:
adata

In [ ]:
annot = []
for n in adata.obs['louvainr02']:
    if n == '0':
        annot.append('0')
    elif n == '1':
        annot.append('1')
adata.obs['low_res_ct'] = annot
del annot

In [ ]:
sc.pl.tsne(adata, color=["Ploidy","low_res_ct"])

Label 4n cells that are in the non-parenchymal cluster

In [ ]:
missorted = []
for i in range(0, len(adata)):
    if adata.obs["low_res_ct"][i] == "0" and adata.obs["Ploidy"][i] == "2n":    
        missorted.append("2n_hepatocytes")
    elif adata.obs["low_res_ct"][i] == "0" and adata.obs["Ploidy"][i] == "4n":    
        missorted.append("4n_hepatocytes")
    elif adata.obs["low_res_ct"][i] == "1" and adata.obs["Ploidy"][i] == "2n": 
        missorted.append("2n_other_cluster")
    elif adata.obs["low_res_ct"][i] == "1" and adata.obs["Ploidy"][i] == "4n": 
        missorted.append("4n_other_cluster")
    else:
        missorted.append("2-2n")
adata.obs["clusters"] = missorted

In [ ]:
sc.pl.tsne(adata, color=['Ploidy', 'clusters'])

Check if missorted cells come from a specific plate

In [ ]:
cf.cell_compo_table(adata, "clusters", "exp.")

### Marker gene expression for cell type annotation

In [ ]:
marker_names_all = []
cell_type_list = []
with open("../cell_type_markers_curated.csv") as f:
    head = f.readline()
    for line in f:
        line = line.split("\t")
        marker_names_all.append(line[0])
        cell_type_list.append(line[1])      
cell_type_dict = {}
for name in list(set(cell_type_list)):
    cell_type_dict[name] = []
for i in range(0, len(cell_type_list)):
    cell_type_dict[cell_type_list[i]].append(marker_names_all[i])
gene_name_dict = {}
for i in range(0, len(adata.var)):
    gene_name_dict[adata.var["gene_name"][i]] = adata.var_names[i]
cell_type_id_dict = {}
for name in list(set(cell_type_list)):
    cell_type_id_dict[name] = []
cell_type_name_dict = {}
for name in list(set(cell_type_list)):
    cell_type_name_dict[name] = []
for name in list(set(cell_type_list)):
    item = cell_type_dict[name]
    for elem in item:
        if elem in gene_name_dict.keys():
            cell_type_id_dict[name].append(gene_name_dict[elem])
            cell_type_name_dict[name].append(elem) 

In [ ]:
# Endothelial cells
endothelial_id = cell_type_id_dict["Endothelial cell"]
endothelial_name = cell_type_name_dict["Endothelial cell"]
sc.pl.tsne(adata, color=endothelial_id, title=endothelial_name, use_raw=True)

In [ ]:
# Cholangiocytes
cholangiocyte_id = cell_type_id_dict['Cholangiocyte']
cholangiocyte_name = cell_type_name_dict['Cholangiocyte']
sc.pl.tsne(adata, color=cholangiocyte_id, title=cholangiocyte_name, use_raw=True)

In [ ]:
# Hepatoblasts
hepatoblast_id = cell_type_id_dict['Hepatoblast']
hepatoblast_name = cell_type_name_dict['Hepatoblast']
sc.pl.tsne(adata, color=hepatoblast_id, title=hepatoblast_name, use_raw=True)

In [ ]:
# Kupffer cells
kupffer_id = cell_type_id_dict['Kupffer cell']
kupffer_name = cell_type_name_dict['Kupffer cell']
sc.pl.tsne(adata, color=kupffer_id, title=kupffer_name, use_raw=True)

In [ ]:
# Dendritic cells
dendritic_id = cell_type_id_dict['Dendritic cell']
dendritic_name = cell_type_name_dict['Dendritic cell']
sc.pl.tsne(adata, color=dendritic_id, title=dendritic_name, use_raw=True)

In [ ]:
# Neutrophils
neutrophil_id = cell_type_id_dict['Neutrophil']
neutrophil_name = cell_type_name_dict['Neutrophil']
sc.pl.tsne(adata, color=neutrophil_id, title=neutrophil_name, use_raw=True)

In [ ]:
# Lymphocytes
lymph_id = cell_type_id_dict["Lymphocyte"]
lymph_name = cell_type_name_dict["Lymphocyte"]
sc.pl.tsne(adata, color=lymph_id, title=lymph_name, use_raw=False)

In [ ]:
# Hepatocytes
hepatocyte_id = cell_type_id_dict['Hepatocyte']
hepatocyte_name = cell_type_name_dict['Hepatocyte']
sc.pl.tsne(adata, color=hepatocyte_id, title=hepatocyte_name, use_raw=True)

In [ ]:
marker_names_all = []
cell_type_list = []
with open("../cell_type_markers_curated.csv") as f:
    head = f.readline()
    for line in f:
        line = line.split("\t")
        marker_names_all.append(line[0])
        cell_type_list.append(line[2][:-1])      
cell_type_dict = {}
for name in list(set(cell_type_list)):
    cell_type_dict[name] = []
for i in range(0, len(cell_type_list)):
    cell_type_dict[cell_type_list[i]].append(marker_names_all[i])
gene_name_dict = {}
for i in range(0, len(adata.var)):
    gene_name_dict[adata.var["gene_name"][i]] = adata.var_names[i]
cell_type_id_dict = {}
for name in list(set(cell_type_list)):
    cell_type_id_dict[name] = []
cell_type_name_dict = {}
for name in list(set(cell_type_list)):
    cell_type_name_dict[name] = []
for name in list(set(cell_type_list)):
    item = cell_type_dict[name]
    for elem in item:
        if elem in gene_name_dict.keys():
            cell_type_id_dict[name].append(gene_name_dict[elem])
            cell_type_name_dict[name].append(elem) 

In [ ]:
# high resolution clustering for the non-parenchymal cells
sc.tl.louvain(adata, resolution=1.2, key_added='louvainr02_12', restrict_to=['louvainr02', ['1']])

sc.pl.pca(adata, wspace=0.4, color=["louvainr02_12", "Ploidy","exp."])
sc.pl.tsne(adata, wspace=0.4, color=["louvainr02_12", "Ploidy","exp."])

Get the top 500 DE genes per louvain cluster and check their overlap with known markers to confirm annotation

In [ ]:
sc.tl.rank_genes_groups(adata, groupby="louvainr02_12", n_genes=500)

In [ ]:
df = sc.tl.marker_gene_overlap(adata, reference_markers=cell_type_id_dict)

In [ ]:
df

In [ ]:
sns.heatmap(df, cmap=sns.cubehelix_palette(69, start=.5, rot=-.75))

How the louvain clusters are annotated:  
- it is clear that cluster 0 are hepatocytes  
- 1,0 has markers for both, endothelial cells and lymphocytes but the overlap with endothelial cells is higher  
- 1,1 has the highest overlap with lymphocyte markers  
- 1,2 also has the highest overlap with lymphocyte markers
- 1,3 are clearly endothelial cells  
- 1,4 are Kupffer and dendritic cells (both macrophages)
- 1,5 has a lot of hepatocyte markers but only half as many as the hepatocyte cluster and also markers for other cell types, suggesting this cluster might not be as finally differentiated as true hepatocytes, so they are labelled hepatobiliary cells
- 1,6 has lymphocyte and hepatocyte markers but is split from the hepatocytes even at low resolution, so they're labelled lymphoytes

In [ ]:
annot = []
for i in range(0, len(adata)):
    if adata.obs["louvainr02_12"][i] in ["0"] and adata.obs["Ploidy"][i] == "2n":
        annot.append("2n Hepatocytes")
    elif adata.obs["louvainr02_12"][i] in ["0"] and adata.obs["Ploidy"][i] == "4n":
        annot.append("4n Hepatocytes")   
    elif adata.obs["louvainr02_12"][i] in ["1,5"]:
        annot.append("Hepatobiliary cells")
    elif adata.obs["louvainr02_12"][i] == "1,2":
        annot.append("Lymphocytes")
    elif adata.obs["louvainr02_12"][i] == "1,1":
        annot.append("Lymphocytes")
    elif adata.obs["louvainr02_12"][i] == "1,6":
        annot.append("Lymphocytes")  
    elif adata.obs["louvainr02_12"][i] == "1,0":
        annot.append("Endothelial cells")
    elif adata.obs["louvainr02_12"][i] == "1,3":
        annot.append("Endothelial cells")   
    elif adata.obs["louvainr02_12"][i] == "1,4":
        annot.append("Kupffer cells and dendritic cells")
    elif adata.obs["louvainr02_12"][i] == "1,6":
        annot.append("Lymphocytes")        
adata.obs["cell_type"] = annot

In [ ]:
sc.pl.tsne(adata, color="cell_type")

In [ ]:
sc.pl.stacked_violin(adata, gene_symbols="gene_name", groupby="cell_type", swap_axes=True, var_names=cell_type_name_dict, save="_markers_cell_type_20200515.pdf")

Combine the hepatocytes and change the order

In [ ]:
annot = []
for elem in adata.obs["cell_type"]:
    if elem in ["2n Hepatocytes", "4n Hepatocytes"]:
        annot.append("Hepatocytes")
    else:
        annot.append(elem)
adata.obs["cell_type1"] = annot  

In [ ]:
annot = []
for elem in adata.obs["cell_type1"]:
    if elem == "Hepatocytes":
        annot.append("01 Hepatocytes")
    elif elem == "Hepatobiliary cells":
        annot.append("02 Hepatobiliary cells")
    elif elem == "Endothelial cells":
        annot.append("03 Endothelial cells")
    elif elem == "Kupffer cells and dendritic cells":
        annot.append("04 Kupffer cells and dendritic cells")
    elif elem == "Lymphocytes":
        annot.append("05 Lymphocytes")
adata.obs["cell_type2"] = annot       

Color the clusters nicely

In [ ]:
adata.uns["low_res_ct_colors"] = ["#686868", "#a9a9a9"]

In [ ]:
adata.uns["cell_type_colors"] = ["#1f77b4", "#ff7f0e", '#bcc2f6', '#251024', '#0b8686', "#ab0dd0"]

In [ ]:
sc.pl.tsne(adata, color="cell_type")

In [ ]:
adata.uns["cell_type2_colors"] = ["#9b0f2b", "#251024", "#bcc2f6", "#006c84", "#9d18af"]

In [ ]:
sc.pl.tsne(adata, color=["Ploidy", "low_res_ct", "cell_type2"])

In [ ]:
adata.uns["louvainr02_12_colors"] = ["#686868", "#b5bd61", "#660066", "#d62728", "#00ccff", "#0b8686", 
                                    "#e377c2", "#FFCC00"]

In [ ]:
sc.pl.tsne(adata, color=["Ploidy", "low_res_ct", "louvainr02_12", "cell_type2"])

Save the annoatated adata

In [ ]:
adata.write('/home/maria/data/polyploid_hepatocytes/matrices/snRNAseq_mouse_hepatocytes_young_cells_filtered_TPM_may20_annotated.h5ad')

In [ ]:
adata = ad.read('/home/maria/data/polyploid_hepatocytes/matrices/snRNAseq_mouse_hepatocytes_young_cells_filtered_TPM_may20_annotated.h5ad')

In [ ]:
# plotting the markers nicely

tiny_marker_dict = {"Hepatocytes":["Cyp27a1","Ppara"],
                    "Hepatobiliary cells":["Sspn","Rplp2"],
                    "Endothelial cells":["Dnase1l3","Egfl7"],
                    "Kupffer and dentritic cells":["Clec4f","Cd5l"],
                    "Lymphocytes":["Bcl2","Lck"]}

small_marker_dict = {"Hepatocytes":["Cyp27a1","Ppara","Pck1"],
                    "Hepatobiliary cells":["Lgr5","Id3","Fgg"],
                    "Endothelial cells":["Dnase1l3","Ptprb","Stab2"],
                    "Kupffer and dentritic cells":["Clec4f","Slc40a1","Cd5l"],
                    "Lymphocytes":["Bcl2","Syk","Lck"]}

bigger_marker_dict = {"Hepatocytes":["Cyp27a1","Ppara","Pck1","Cyp2d26","Cps1",
                                    "Ces3a","Cyp3a25","Neat1","Abcc2"],
                    "Hepatobiliary cells":["Lgr5","Id3","Hnf1b","Glul","Gas1",
                                          "Filip1l","Sspn","Trf","Rplp2"],
                    "Endothelial cells":["Dnase1l3","Ptprb","Stab2","Egfl7","Ehd3",
                                        "Clec4g","Fcgr2b","Calcrl","Plekhg1"],
                    "Kupffer and dentritic cells":["Clec4f","Slc40a1","Cd5l","Ctsb","H2-Aa",
                                                  "Dmpk","Slc9a9","Zeb2","Slc8a1"],
                    "Lymphocytes":["Bcl2","Syk","Lck","Edem1","Bcl11b",
                                  "Usp14","Lix1","Nras","Cdc7"]}

sc.pl.stacked_violin(adata, gene_symbols="gene_name", groupby="cell_type2", var_names=tiny_marker_dict, 
                     standard_scale="var")

In [ ]:
markers = [item for sublist in bigger_marker_dict.values() for item in sublist]

In [ ]:
sc.pl.tsne(adata, color=markers, gene_symbols="gene_name")

#### For the heatmap in R

For better visualisation, remove rRNA genes

In [ ]:
adatar = ad.read("adata_rRNA.h5ad")
rrna = []
for elem in adata.var_names.tolist():
    if elem in adatar.var_names.tolist():
        rrna.append("yes")
    else:
        rrna.append("no")
adata.var["rrna"] = rrna

In [ ]:
adatam = adata[:,adata.var["rrna"] == "no"]

Subsample cells

In [ ]:
adata1 = adatam[adatam.obs["cell_type1"] == "Hepatocytes",:].copy()
sc.pp.subsample(adata1, n_obs=40, random_state=1)
adata2 = adatam[adatam.obs["cell_type1"] == "Hepatobiliary cells",:].copy()
sc.pp.subsample(adata2, n_obs=40, random_state=1)
adata3 = adatam[adatam.obs["cell_type1"] == "Endothelial cells",:].copy()
sc.pp.subsample(adata3, n_obs=40, random_state=1)
adata4 = adatam[adatam.obs["cell_type1"] == "Kupffer cells and dendritic cells",:].copy()
sc.pp.subsample(adata4, n_obs=40, random_state=1)
adata5 = adatam[adatam.obs["cell_type1"] == "Lymphocytes",:].copy()
sc.pp.subsample(adata5, n_obs=40, random_state=1)

In [ ]:
adataS = adata1.concatenate(adata2, adata3, adata4, adata5)

In [ ]:
sc.tl.rank_genes_groups(adataS, groupby="cell_type2", n_genes=20, layer="raw", use_raw=False)

In [ ]:
unsup_markers = []
for elem in list(adataS.uns["rank_genes_groups"]["names"]['01 Hepatocytes'])[0:12]:
    unsup_markers.append(elem)
for elem in list(adataS.uns["rank_genes_groups"]["names"]['02 Hepatobiliary cells'])[0:12]:
    unsup_markers.append(elem)
for elem in list(adataS.uns["rank_genes_groups"]["names"]['03 Endothelial cells'])[0:12]:
    unsup_markers.append(elem)
for elem in list(adataS.uns["rank_genes_groups"]["names"]['04 Kupffer cells and dendritic cells'])[0:12]:
    unsup_markers.append(elem)
for elem in list(adataS.uns["rank_genes_groups"]["names"]['05 Lymphocytes'])[0:12]:
    unsup_markers.append(elem)    

In [ ]:
sc.pl.heatmap(adataS, groupby="cell_type2", var_names=unsup_markers, standard_scale="var")

To have ploidy as additional annotation, we do the heatmap in R

In [ ]:
adatax = adataS[:,unsup_markers]

In [ ]:
np.savetxt("/home/maria/data/polyploid_hepatocytes/cell_type_markers_mat_new.txt", adatax.layers["raw"], delimiter="\t", fmt='%.8f')
adatax.write_csvs("/home/maria/data/polyploid_hepatocytes/metadata/cnvs/cell_type_markers_new/")

In [ ]:
sc.pl.matrixplot(adata, gene_symbols="gene_name", groupby="cell_type2", 
                 var_names=bigger_marker_dict, use_raw=False, standard_scale="var")   

In [ ]:
sc.pl.correlation_matrix(adata, groupby="cell_type", save="_cell_types_20200529.pdf")

### Cell cycle analysis

In [ ]:
import rpy2.rinterface
import logging

from rpy2.robjects import pandas2ri
import anndata2ri
%load_ext rpy2.ipython
anndata2ri.activate()

In [ ]:
data_mat = adata.X.T
gnames = adata.var_names.tolist()
cnames = adata.obs_names.tolist()

In [ ]:
%%R -i data_mat -i gnames -i cnames -o phases -o scores

library(scran)
# cell cycle analysis using cyclone
# should be easy because it's implemented in scran
mm.pairs <- readRDS(system.file("exdata", "mouse_cycle_markers.rds", package="scran"))

rownames(data_mat) <- gnames
colnames(data_mat) <- cnames

score_list <- cyclone(data_mat, mm.pairs, gene.names=rownames(data_mat), iter=1000, min.iter=100, min.pairs=50,  BPPARAM=bpparam(), verbose=FALSE)

phases <- score_list$phases
scores <- score_list$scores

In [ ]:
adata.obs["cyclone_phases"] = phases
adata.obs["score_G1"] = scores["G1"].tolist()
adata.obs["score_S"] = scores["S"].tolist()
adata.obs["score_G2M"] = scores["G2M"].tolist()

In [ ]:
adata.uns["cyclone_phases_colors"] = ['#505050', '#bc086b', '#0af9ea']

In [ ]:
sc.pl.tsne(adata, color=["cyclone_phases","Ploidy", "score_S", "score_G1", "score_G2M"])

In [ ]:
sc.pl.tsne(adata, color=["cyclone_phases","Ploidy", "score_G1"], wspace=0.3, save="_Figure2_B_20200515.pdf")

In [ ]:
cf.cell_compo_table(adata,"clusters", "cyclone_phases")

In [ ]:
cf.cell_compo_table(adata,"cell_type", "cyclone_phases")

In [ ]:
# Data
df = cf.cell_compo_table(adata,"cyclone_phases", "clusters")
r = np.arange(5)
 
# From raw value to percentage
totals = [i+j+k for i,j,k in zip(df['G1'], df['G2M'], df['S'])]

G1 = [i / j * 100 for i,j in zip(df['G1'], totals)]
G2M = [i / j * 100 for i,j in zip(df['G2M'], totals)]
S = [i / j * 100 for i,j in zip(df['S'], totals)]

# plot
barWidth = 0.85
names = ('2n_hep', '2n_other', "4n_hep", "4n_other")

# Create bars
plt.bar(r, G1, width=barWidth, label="G1", color=("#505050"))
plt.bar(r, G2M, bottom=G1, width=barWidth, label="G2M", color=("#bc086b"))
plt.bar(r, S, bottom=[i+j for i,j in zip(G1, G2M)], width=barWidth, label="S", color=("#0af9ea"))


    
# Custom x axis
plt.xticks(r, names)
#plt.xlabel("group")
plt.xticks(rotation=90)
plt.legend(loc='upper left', bbox_to_anchor=(1,1), ncol=1)

# Show graphic
#plt.show()
plt.savefig("barplot_cell_cycle_20200518.pdf")

### Differential expression between cell types

In [ ]:
sc.tl.rank_genes_groups(adata, groupby="cell_type1", n_genes=19258, method="t-test")

In [ ]:
dict_genes = adata.uns["rank_genes_groups"].copy()

In [ ]:
df = {}
for idx, elem in enumerate(list(dict_genes["names"].dtype.names)):
    print(idx, elem)
    df[elem+"_gene"] = []
    df[elem+"_logfoldchange"] = []
    df[elem+"_pval_adj"] = []
    for j in range(0, len(adata.var)):
        df[elem+"_gene"].append(dict_genes["names"][j][idx])
        df[elem+"_logfoldchange"].append(dict_genes["logfoldchanges"][j][idx])
        df[elem+"_pval_adj"].append(dict_genes["pvals_adj"][j][idx])

dataframe = pd.DataFrame(df)        
for idx, elem in enumerate(list(dict_genes["names"].dtype.names)):
    adata.var[elem+"_mean"] = np.mean(adata[adata.obs["cell_type1"] == elem,:].layers["norm_counts"], axis=0)
    adata.var[elem+"_n_cells"] = np.sum(adata[adata.obs["cell_type1"] == elem,:].layers["norm_counts"] > 0, axis=0)
    dataframe.index = dataframe[elem+"_gene"]
    adata.var[elem+"_log2FC"] = dataframe[elem+"_logfoldchange"]
    adata.var[elem+"_pvals_adj"] = dataframe[elem+"_pval_adj"]

In [ ]:
adata.var

### Differential expression between 2n and 4n hepatocytes

In [ ]:
adata1 = adata[adata.obs["cell_type1"] == "Hepatocytes",:].copy()

In [ ]:
sc.tl.rank_genes_groups(adata1, groupby="cell_type", n_genes=19258, method="t-test")

In [ ]:
dict_genes = adata1.uns["rank_genes_groups"].copy()

In [ ]:
df = {}
for idx, elem in enumerate(list(dict_genes["names"].dtype.names)):
    print(idx, elem)
    df[elem+"_gene"] = []
    df[elem+"_logfoldchange"] = []
    df[elem+"_pval_adj"] = []
    for j in range(0, len(adata.var)):
        df[elem+"_gene"].append(dict_genes["names"][j][idx])
        df[elem+"_logfoldchange"].append(dict_genes["logfoldchanges"][j][idx])
        df[elem+"_pval_adj"].append(dict_genes["pvals_adj"][j][idx])

dataframe = pd.DataFrame(df)        
for idx, elem in enumerate(list(dict_genes["names"].dtype.names)):
    adata.var[elem+"_mean"] = np.mean(adata[adata.obs["cell_type"] == elem,:].layers["norm_counts"], axis=0)
    #adata.var[elem+"_sum"] = np.sum(adata[adata.obs["cell_type"] == elem,:].layers["norm_counts"], axis=0)
    adata.var[elem+"_n_cells"] = np.sum(adata[adata.obs["cell_type"] == elem,:].layers["norm_counts"] > 0, axis=0)
    dataframe.index = dataframe[elem+"_gene"]
    adata.var[elem+"_log2FC"] = dataframe[elem+"_logfoldchange"]
    adata.var[elem+"_pvals_adj"] = dataframe[elem+"_pval_adj"]

In [ ]:
adata.var

In [ ]:
changes = []
for j in range(0, len(adata.var)):
    if adata.var["2n Hepatocytes_log2FC"][j] > 0.5 and adata.var["2n Hepatocytes_pvals_adj"][j] < 0.05:
        changes.append("up_2n")
    elif adata.var["4n Hepatocytes_log2FC"][j] > 0.5 and adata.var["4n Hepatocytes_pvals_adj"][j] < 0.05:
        changes.append("up_4n")
    else:
        changes.append("none")
        
adata.var["changes_hep"] = changes        

In [ ]:
adata.var.to_csv("gene_expression_cell_types_and_ploidy_20200522_filter25.csv")

#### Write DE genes per cell type to file

In [ ]:
DE_dict = {}
for elem in list(set(adata.obs["cell_type"])): # to get the hepatocytes together, re-run with "cell_type1"
    DE_dict[elem+"_up"] = 0
    DE_dict[elem+"_down"] = 0
    outfile_up = open("differential_expression/up_"+elem, "w")
    outfile_down = open("differential_expression/down_"+elem, "w")
    for j in range(0,len(adata.var)):
        if adata.var[elem+"_log2FC"][j] > 0.5 and adata.var[elem+"_pvals_adj"][j] < 0.05:
            DE_dict[elem+"_up"] += 1
            outfile_up.write(adata.var_names[j])
            outfile_up.write("\n")
        elif adata.var[elem+"_log2FC"][j] < -0.5 and adata.var[elem+"_pvals_adj"][j] < 0.05:
            DE_dict[elem+"_down"] += 1
            outfile_down.write(adata.var_names[j])
            outfile_down.write("\n")
    outfile_up.close()
    outfile_down.close()

### Plotting DE genes between 2n and 4n hepatocytes

In [ ]:
cols = []
for elem in adata.var["changes_hep"]:
    if elem == "up_2n":
        cols.append("#1f77b4")
    elif elem == "up_4n":
        cols.append("#ff7f0e")
    else:
        cols.append("0.75")

In [ ]:
fig = plt.scatter(y=-np.log10(adata.var["2n Hepatocytes_pvals_adj"]), x=adata.var["2n Hepatocytes_log2FC"],
                  c=cols,alpha=0.5)
#plt.savefig("volcano_DE_4n_2n_20200515.pdf")

Get the number of genes per ploidy condition

In [ ]:
genes2n = pd.DataFrame()
genes4n = pd.DataFrame()
adata2n = adata[adata.obs["cell_type"] == "2n Hepatocytes"].copy()
adata4n = adata[adata.obs["cell_type"] == "4n Hepatocytes"].copy()

genes2n["n_genes_2n"] = adata2n.obs["n_genes"]
genes4n["n_genes_4n"] = adata4n.obs["n_genes"]

Small dataframe with means and number of cells in which a gene is expressed

In [ ]:
df = pd.DataFrame()
df["mean_2n"] = adata.var["2n Hepatocytes_mean"]
df["mean_4n"] = adata.var["4n Hepatocytes_mean"]
df["cells_2n"] = adata.var["2n Hepatocytes_n_cells"]
df["cells_4n"] = adata.var["4n Hepatocytes_n_cells"]
df["changes"] = adata.var["changes_hep"]
df.index = adata.var.index

Removing genes with low or high mean for visualisation purposes

In [ ]:
df1 = df[df["mean_2n"] > 0.1]
df1 = df1[df1["mean_2n"] < 100]
df2 = df1[df1["mean_4n"] > 0.1]
df2 = df2[df2["mean_2n"] < 100]

In [ ]:
up2n = df[df["changes"] == "up_2n"]

In [ ]:
up4n = df[df["changes"] == "up_4n"]

#### Plotting mean 4n vs mean 2n, highlighting DE genes and including number of cells in which DE genes are expressed

In [ ]:
from scipy.stats import gaussian_kde

# colors for scatter plot
cols = []
for elem in df2["changes"]:
    if elem == "up_2n":
        cols.append("#1f77b4")
    elif elem == "up_4n":
        cols.append("#ff7f0e")
    else:
        cols.append("0.85")

# definitions for the axes
left, width = 0.1, 0.65
bottom, height = 0.1, 0.65
spacing = 0.02


rect_scatter = [left, bottom, width, height]
rect_histx = [left, bottom + height + spacing, width, 0.2]
rect_histy = [left + width + spacing, bottom, 0.22, height]


# start with a rectangular Figure
plt.figure(figsize=(8, 8))

ax_scatter = plt.axes(rect_scatter)
#plt.xscale("log")
#plt.yscale("log")
#ident = [0.0, 1000]
#plt.plot(ident,ident, color="k")
ax_scatter.tick_params(direction='in', top=True, right=True)
ax_histx = plt.axes(rect_histx)
ax_histx.tick_params(direction='in', labelbottom=False)
ax_x = plt.axvline(up2n["cells_2n"].mean(), color='k', linestyle='dashed', linewidth=1.5)
ax_histy = plt.axes(rect_histy)
ax_histy.tick_params(direction='in', labelleft=False)
ax_y = plt.axhline(up4n["cells_4n"].mean(), color='k', linestyle='dashed', linewidth=1.5)


# the scatter plot:
ax_scatter.scatter(x=df2["cells_2n"], y=df2["cells_4n"], c=cols, alpha=0.5)

# now determine nice limits by hand:
binwidth = 10
lim = np.ceil(np.abs([df2["cells_2n"], df2["cells_4n"]]).max() / binwidth) * binwidth
ax_scatter.set_xlim((0, lim))
ax_scatter.set_ylim((0, lim))

bins = np.arange(0, lim + binwidth, binwidth)
dist_space = np.linspace(0, lim, 1000)

# using gaussian_kde to get a density plot instead of histogram
density2n = gaussian_kde(up2n["cells_2n"])
density4n = gaussian_kde(up4n["cells_4n"], bw_method=.1)


ax_histx.plot(dist_space, density2n(dist_space), c="#1f77b4")
ax_x
ax_histy.plot(density4n(dist_space), dist_space, c="#ff7f0e")
ax_y


#ax_histx.hist(up2n["cells_2n"], bins=bins, histtype="stepfilled", density=True)
#ax_histy.hist(up4n["cells_4n"], bins=bins, orientation='horizontal', histtype="stepfilled", density=True)

ax_histx.set_xlim(ax_scatter.get_xlim())
ax_histy.set_ylim(ax_scatter.get_ylim())
ax_histy.set_xlim(ax_histx.get_ylim())
ax_histy.set_xticks(ticks=[0.000,0.002,0.004,0.006])
ax_histy.set_xticklabels(labels=[0.000,None,None,0.006])


#plt.savefig("scatter_density_Figure3_D_ncells_for_DE_genes_removed_nonDE_in_scatter_20200529.pdf", bbox_inches = "tight")

In [ ]:
adata1[adata1.obs["Ploidy"] == "2n"]

In [ ]:
adata1[adata1.obs["Ploidy"] == "4n"]

How many 2n nuclei express the genes that are upregulated in 4n?

In [ ]:
np.mean(up4n["cells_2n"])/320

How many 4n nuclei express the genes that are upregulated in 2n?

In [ ]:
np.mean(up2n["cells_4n"])/741

### Gene set enrichment of up-regulated genes per cell type

In [ ]:
from gprofiler import gprofiler

Here is one example:

In [ ]:
plt.rcParams['figure.figsize']=(10,8) #rescale figures
genes = []
with open("differential_expression/up_4n Hepatocytes") as f:
    head = f.readline()
    for line in f:
        genes.append(line[:-1])
    
#Interpretation of differentially expressed genes in paneth cells - g:profiler
gp = gprofiler(genes, organism='mmusculus')
gp = gp[gp["domain"] == "BP"]

gp_enrichment = gp.sort_values('p.value').iloc[:,[2,3,5,6,11]]
gp_enrichment['name']= gp_enrichment['term.name'].copy()
gp_enrichment['p_value']= gp_enrichment['p.value'].copy()
gp_enrichment['intersection_size']= gp_enrichment['overlap.size'].copy()
del gp_enrichment['term.name'], gp_enrichment['overlap.size'], gp_enrichment['p.value']
cf.plot_enrich(gp_enrichment)#, save='GO_4n_vs_2n_true_DE_20200515.pdf')

### Coefficient of variation

Change figure parameters back to normal

In [ ]:
sc.settings.set_figure_params(dpi=80)

sc.set_figure_params(scanpy=True, dpi=80, dpi_save=250,
                     frameon=True, vector_friendly=True,
                     color_map="YlGnBu", format='pdf', transparent=False,
                     ipython_format='png2x')

In [ ]:
adata1 = adata.copy()

In [ ]:
adata1.var["mean_norm_counts"] = np.mean(adata1.layers["raw"], axis=0)
adata1.var["mean_norm_counts_log"] = np.mean(adata1.X, axis=0)

In [ ]:
# remove genes with low mean expression
adata1 = adata1[:,adata1.var["mean_norm_counts"] > 0.25].copy()

In [ ]:
df = pd.DataFrame()
df2 = pd.DataFrame()
for elem in ['Hepatocytes', 'Kupffer cells and dendritic cells', 'Endothelial cells',
             'Hepatobiliary cells', 'Lymphocytes']:
    adata1.var["mean_"+elem] = np.mean(adata1[adata1.obs["cell_type1"] == elem,:].layers["norm_counts"], axis=0)
    adata1.var["CV_"+elem] = np.sqrt(np.exp(np.std(adata1[adata1.obs["cell_type1"] == elem,:].X, axis=0)**2)-1)
    df["CV_"+elem] = np.sqrt(np.exp(np.std(adata1[adata1.obs["cell_type1"] == elem,:].X, axis=0)**2)-1)
    df2["mean_"+elem] = np.mean(adata1[adata1.obs["cell_type1"] == elem,:].layers["norm_counts"], axis=0)

In [ ]:
p = df.boxplot()
plt.xticks(rotation=90)
plt.yscale("log")
plt.savefig("boxplot_Figure3_E_variability_all.pdf")

In [ ]:
df = pd.DataFrame()
df2 = pd.DataFrame()
for elem in ['2n Hepatocytes', '4n Hepatocytes']:
    adata1.var["mean_"+elem] = np.mean(adata1[adata1.obs["cell_type"] == elem,:].layers["norm_counts"], axis=0)
    adata1.var["CV_"+elem] = np.sqrt(np.exp(np.std(adata1[adata1.obs["cell_type"] == elem,:].X, axis=0)**2)-1)
    df["CV_"+elem] = np.sqrt(np.exp(np.std(adata1[adata1.obs["cell_type"] == elem,:].X, axis=0)**2)-1)
    df2["mean_"+elem] = np.mean(adata1[adata1.obs["cell_type"] == elem,:].layers["norm_counts"], axis=0)

In [ ]:
q = df.boxplot()
plt.xticks(rotation=90)
q.set_ylim(p.get_ylim())
plt.yscale("log")
plt.savefig("boxplot_Figure3_E_variability_hep.pdf")

In [ ]:
stats.mannwhitneyu(df["CV_2n Hepatocytes"].dropna(), df["CV_4n Hepatocytes"].dropna())

In [ ]:
np.median(df["CV_2n Hepatocytes"])/np.median(df["CV_4n Hepatocytes"])

### Highly variable genes per cell type

In [ ]:
for elem in list(set(adata1.obs["cell_type"])):
    outfile=open("differential_variability/high_CV_"+elem+".txt","w")
    for idx, value in enumerate(adata1.var["CV_"+elem].tolist()):
        if value > 1.5:
            outfile.write(adata1.var_names.tolist()[idx])
            outfile.write("\n")
    outfile.close()

### Stem cell markers

In [ ]:
adata1 = adata[adata.obs["cell_type1"] == "Hepatocytes"]

In [ ]:
HSC_name = []
HSC_id = []
index = 0
for n in adata1.var["gene_name"]:
    if n in ["Afp","Epcam","Icam1","Itga6","Lgr5","Notch2","Prom1","Sox9","Tbx3","Tert", "Axin2"] :
        HSC_name.append(n)
        HSC_id.append(adata1.var_names[index])
    index += 1
sc.pl.heatmap(adata1,groupby="clusters", var_names=HSC_id, use_raw=False, show_gene_labels=True,
              standard_scale="obs",swap_axes=True)

In [ ]:
annot = []
for elem in adata.obs["cell_type1"]:
    if elem in ["Hepatocytes", "Hepatobiliary cells"]:
        annot.append("hep-like")
    else:
        annot.append("non-parenchymal")
adata.obs["rough_annot"] = annot       

In [ ]:
adata2 = adata[adata.obs["rough_annot"] == "hep-like"].copy()

In [ ]:
sc.pl.heatmap(adata2 ,groupby="cell_type", var_names=HSC_id, use_raw=False, show_gene_labels=True,
              standard_scale="obs",swap_axes=True)

#### Stem cell marker coexpression in hepatocytes

In [ ]:
adata3 = adata1[:,HSC_id].copy()

In [ ]:
sc.pp.filter_cells(adata3, min_counts=1)

In [ ]:
adata3.X = adata3.layers["raw_bin"]

In [ ]:
sc.pl.clustermap(adata3, use_raw=False, obs_keys="Ploidy", cmap=sns.cubehelix_palette(20), figsize=(15,15), 
                 method="average", metric="jaccard", standard_scale=0)

In [ ]:
adata4 = adata3[adata3.obs["Ploidy"] == "2n",:].copy()
adata5 = adata3[adata3.obs["Ploidy"] == "4n",:].copy()

In [ ]:
from scipy.cluster.hierarchy import linkage
df = pd.DataFrame(adata3.layers["raw_bin"].T) # genes are rows, cells are columns now
df.index = adata3.var["gene_name"] # add gene names as row names
df.columns = adata3.obs.index # add cell names as column names

lut = dict(zip(adata3.obs["Ploidy"], "b"))
col_colors = adata3.obs["Ploidy"].map(lut)

gene_link = linkage(df, metric="jaccard")
cell_link = linkage(df.transpose(), metric="jaccard")

gene_link.shape

g = sns.clustermap(df, col_colors=col_colors, col_linkage=cell_link, row_linkage=gene_link, cmap=sns.cubehelix_palette(20), standard_scale=1, figsize=(15,15))
#g.savefig("heatmap_Figure4_B_20200515.pdf")

In [ ]:
df = pd.DataFrame(adata4.layers["raw_bin"].T) # genes are rows, cells are columns now
df.index = adata4.var["gene_name"] # add gene names as row names
df.columns = adata4.obs.index # add cell names as column names

lut = dict(zip(adata3.obs["Ploidy"], "b"))
col_colors = adata3.obs["Ploidy"].map(lut)

gene_link = linkage(df, metric="jaccard")
cell_link = linkage(df.transpose(), metric="jaccard")

gene_link.shape

g = sns.clustermap(df, col_colors=col_colors, col_linkage=cell_link, row_linkage=gene_link, cmap=sns.cubehelix_palette(20), standard_scale=1, figsize=(15,15))
#g.savefig("heatmap_Figure4_B_2n_20200515pdf")

In [ ]:
df = pd.DataFrame(adata5.layers["raw_bin"].T) # genes are rows, cells are columns now
df.index = adata5.var["gene_name"] # add gene names as row names
df.columns = adata5.obs.index # add cell names as column names

lut = dict(zip(adata3.obs["Ploidy"], "b"))
col_colors = adata3.obs["Ploidy"].map(lut)

gene_link = linkage(df, metric="jaccard")
cell_link = linkage(df.transpose(), metric="jaccard")

gene_link.shape

g = sns.clustermap(df, col_colors=col_colors, col_linkage=cell_link, row_linkage=gene_link, cmap=sns.cubehelix_palette(20), standard_scale=1, figsize=(15,15))
#g.savefig("heatmap_Figure4_B_4n_20200515.pdf")

In [ ]:
def seriation(Z,N,cur_index):
    '''
        input:
            - Z is a hierarchical tree (dendrogram)
            - N is the number of points given to the clustering process
            - cur_index is the position in the tree for the recursive traversal
        output:
            - order implied by the hierarchical tree Z
            
        seriation computes the order implied by a hierarchical tree (dendrogram)
    '''
    if cur_index < N:
        return [cur_index]
    else:
        left = int(Z[cur_index-N,0])
        right = int(Z[cur_index-N,1])
        return (seriation(Z,N,left) + seriation(Z,N,right))
    
def compute_serial_matrix(dist_mat,method="ward"):
    '''
        input:
            - dist_mat is a distance matrix
            - method = ["ward","single","average","complete"]
        output:
            - seriated_dist is the input dist_mat,
              but with re-ordered rows and columns
              according to the seriation, i.e. the
              order implied by the hierarchical tree
            - res_order is the order implied by
              the hierarhical tree
            - res_linkage is the hierarhical tree (dendrogram)
        
        compute_serial_matrix transforms a distance matrix into 
        a sorted distance matrix according to the order implied 
        by the hierarchical tree (dendrogram)
    '''
    N = len(dist_mat)
    flat_dist_mat = squareform(dist_mat)
    res_linkage = linkage(flat_dist_mat, method=method)
    res_order = seriation(res_linkage, N, N + N-2)
    seriated_dist = np.zeros((N,N))
    a,b = np.triu_indices(N,k=1)
    seriated_dist[a,b] = dist_mat[ [res_order[i] for i in a], [res_order[j] for j in b]]
    seriated_dist[b,a] = seriated_dist[a,b]
    
    return seriated_dist, res_order, res_linkage



In [ ]:
df = pd.DataFrame(adata3.layers["raw_bin"].T)
df.index = adata3.var["gene_name"]
df.columns = adata3.obs.index

In [ ]:
from scipy.spatial.distance import pdist, squareform

In [ ]:
x = pdist(df, metric="jaccard")
x2 = squareform(x)

In [ ]:
ordered_dist_mat, res_order, res_linkage = compute_serial_matrix(x2,"ward")

In [ ]:
df = pd.DataFrame(ordered_dist_mat)
df.index = adata3.var["gene_name"][res_order]
df.columns = adata3.var["gene_name"][res_order]
mask = np.triu(np.ones_like(df, dtype=np.bool))
sns.heatmap(df, cmap="RdBu")#, mask=mask)
#plt.savefig("heatmap_jaccard_distance_stem_cell_markers_2n_20200519.pdf")

### Pseudospacial ordering based on zonation markers

In [ ]:
adata1 = adata[adata.obs["cell_type1"] == "Hepatocytes"]

In [ ]:
sc.pl.violin(adata1, "n_genes", groupby="Ploidy")

In [ ]:
np.median(adata1[adata1.obs["Ploidy"] == "4n"].obs["n_genes"])/np.median(adata1[adata1.obs["Ploidy"] == "2n"].obs["n_genes"])

In [ ]:
sc.pl.scatter(adata1, x='n_counts_TPM', y='n_genes', color='Ploidy')

In [ ]:
sc.tl.tsne(adata1, perplexity=15, n_pcs=15)
sc.pl.tsne(adata1, color="Ploidy")

In [ ]:
df = pd.read_csv('../../../zonation_markers.txt', sep='\t')
zonation_markers = []
for marker in df.MouseGeneID.tolist():
    if ';' in marker:
        zonation_markers += marker.split(';')
    else:
        zonation_markers.append(marker)

In [ ]:
df = pd.read_csv('../../../zonation_markers_new.txt', sep='\n')
zonation_markers1 = []
for marker in df.MouseGeneID.tolist():
    if ';' in marker:
        zonation_markers1 += marker.split(';')
    else:
        zonation_markers1.append(marker)

In [ ]:
zonation_markers_all = zonation_markers+zonation_markers1

In [ ]:
zonation_markers2 = []
for idx,gene in enumerate(list(adata1.var['gene_name'])):
    if gene in zonation_markers_all:
        zonation_markers2.append(list(adata1.var_names)[idx])

In [ ]:
adata2 = adata1[:,zonation_markers2].copy()
adata2

In [ ]:
sc.pp.pca(adata2)
sc.pp.neighbors(adata2)
sc.tl.umap(adata2)
sc.tl.diffmap(adata2)

In [ ]:
sc.pl.umap(adata2, color='Ploidy')
sc.pl.diffmap(adata2, color=['Ploidy',"cell_type", "n_genes"])

In [ ]:
sc.tl.louvain(adata2, key_added="louvainr1", resolution=1)
sc.tl.louvain(adata2, key_added="louvainr05", resolution=0.5)

In [ ]:
adata2.uns["louvainr05_colors"] = ['#4f38b7', '#74489d', '#2498e2']

In [ ]:
sc.pl.diffmap(adata2, color=["louvainr05", 'Ploidy', "n_counts_transcripts", "n_genes"])

In [ ]:
annot = []
for elem in adata2.obs["louvainr05"]:
    if elem in ["0","1"]:
        annot.append("periportal")
    else:
        annot.append("pericentral")
adata2.obs["pseudospace"] = annot   

In [ ]:
sc.pl.diffmap(adata2, color=["ENSMUSG00000017950","ENSMUSG00000025479","ENSMUSG00000029368"], 
              title=["Hnf4a","Cyp2e1", "Alb"])

In [ ]:
sc.pl.diffmap(adata2, color=["louvainr05", 'Ploidy'])

In [ ]:
cf.cell_compo_table(adata2, "louvainr05","Ploidy")

In [ ]:
# Data
df = cf.cell_compo_table(adata2,"Ploidy", "pseudospace")
r = np.arange(3)
 
# From raw value to percentage
totals = [i+j for i,j in zip(df['2n'], df['4n'])]
hep2n = [i / j * 100 for i,j in zip(df['2n'], totals)]
hep4n = [i / j * 100 for i,j in zip(df['4n'], totals)]

# plot
barWidth = 0.85
names = ('pericentral', 'periportal')

# Create bars
plt.bar(r, hep2n, width=barWidth, label="2n")
plt.bar(r, hep4n, bottom=hep2n, width=barWidth, label="4n")

    
# Custom x axis
plt.xticks(r, names)
#plt.xlabel("group")
plt.xticks(rotation=90)
plt.legend(loc='upper left', bbox_to_anchor=(1,1), ncol=1)

#plt.show()
#plt.savefig("barplot_Figure5_D_20200515.pdf")

In [ ]:
adata2.uns["pseudospace_colors"] = ['#6e0451', '#2498e2']

In [ ]:
adata2.uns['iroot'] = 561
sc.tl.dpt(adata2)
sc.tl.paga(adata2, groups="pseudospace")

In [ ]:
sc.tl.rank_genes_groups(adata2, groupby="pseudospace")

In [ ]:
unsup_markers = []
for elem in list(adata2.uns["rank_genes_groups"]["names"]['pericentral'])[0:30]:
    unsup_markers.append(elem)
for elem in list(adata2.uns["rank_genes_groups"]["names"]['periportal'])[0:30]:
    unsup_markers.append(elem)

Zonation marker genes from Halpern et al, 2017

In [ ]:
bahar_genes = []
for gene in ["Glul","Cyp2e1","Cyp2f2","Asl","Ass1","Alb","Cyp1a2","Rnase4","Igfbp1","Gsta3","Ugt1a1",
            "Cyp27a1","Glud1","Cyp8b1","Hamp","Igfbp2","Mup3","Pck1","Cps1","Arg1","Apoa1","G6pc",
            "Uox","Igf1","Pigr","Acly","Axin2","Gstm","Psmd4","C2","Sdhd"]:
    if gene in adata2.var["gene_name"].tolist():
        bahar_genes.append(gene)
sc.pl.diffmap(adata2, color=bahar_genes, gene_symbols="gene_name")

In [ ]:
for gene in bahar_genes:
    df = pd.DataFrame()
    df["dpt"] = adata2.obs["dpt_pseudotime"]
    df["expression"] = np.array(adata2[:,adata2.var["gene_name"]==gene].X)
    bins = [0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1]
    labels = [0,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1]
    df['zones'] = pd.cut(df['dpt'], bins=bins, labels=labels)
    df2 = df.groupby(df["zones"]).mean()     
    
    plt.figure()
    plt.plot(df2.index.tolist(), df2["expression"])
    plt.ylabel('mean_expression')
    plt.title(gene)
    plt.xlabel('pseudospace_bins')
    plt.savefig("zonation_"+gene+".pdf", bbox_inches="tight")

### PAGA path

In [ ]:
plt.rcParams['figure.figsize']=(12,12) #rescale figures
sc.pl.paga_path(adata2, nodes=['pericentral', 'periportal'], keys=unsup_markers, normalize_to_zero_one=True)

#### PAGA for cell types

In [ ]:
sc.tl.paga(adata, groups="cell_type")
sc.tl.draw_graph(adata)

In [ ]:
plt.rcParams['figure.figsize']=(5,5) #rescale figures
sc.pl.paga(adata, color="cell_type")

In [ ]:
plt.rcParams['figure.figsize']=(5,5) #rescale figures
sc.pl.draw_graph(adata, color="cell_type")